In [7]:
import pandas as pd
import warnings
import re
import requests

from datetime import datetime
from time import sleep
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

warnings.simplefilter('ignore')

In [8]:
__BASE_URL__ = 'https://auto.drom.ru/all/page'


def get_items_url(i):
    # генерирует ссылку для получения списка машин
    return f"{__BASE_URL__}{i}/"

In [9]:
def parse_items(html_source):
    # парсит ссылки на отдельные машины
    items = list()
    soup = BeautifulSoup(html_source, 'html.parser')
    for item in soup.find_all('a', attrs={'name': re.compile('[0-9]+'), 'class': 'b-advItem'}):
        items.append(item.attrs['href'])
    return items

In [22]:
def parse_item(html_source):
    #Парсит информацию о машине
    car = {}
    maproperty = {
        'Двигатель': re.compile('Двигатель:\| ([а-яА-Я\, 0-9\.]+).+\|'),
        'Мощность': re.compile('Мощность:[\| ]+\|(\d+)'),
        'Трансмиссия': re.compile('Трансмиссия:[\| ]+([а-яА-я]+)\|'),
        'Привод': re.compile('Привод:[\| ]+([а-яА-я\w]+)\|'),
        'Пробег': re.compile('Пробег, км:[\| ]+(\d+)\|'),
        'Руль': re.compile('Руль:[\| ]+([а-яА-я]+)'),
    }
    soup = BeautifulSoup(html_source, 'html.parser')
    price = soup.find('div', attrs={'color': 'red'})
    price = re.compile('^([0-9]+).+').findall(
        price.get_text().replace('\xa0', '')
    )
    if len(price) > 0:
        car['Цена'] = price[0]
    else:
        car['Цена'] = ''
    car['Год'] = re.compile('([0-9]{4}).*год').findall(
        soup.find('h1', attrs={'class': 'b-title_type_h1'}).get_text()
    )[0]

    car['Модель'] = re.compile('Продажа ([a-zА-Я 0-9\-]+),', re.IGNORECASE).findall(
        soup.find('h1', attrs={'data-ga-stats-ya-name': 'sales_view_bull'}).text)[0]

    raw_mark = soup.find('div', attrs={'class': 'b-media-cont b-media-cont_relative'}).get_text('|')
    for mark in maproperty.keys():
        finded = maproperty[mark].findall(raw_mark)
        if len(finded) > 0:
            car[mark] = finded[0]
        else:
            car[mark] = ''
    return car

In [23]:
def get_data(count_pages=1):
    # запускает парсер и возвращает масив словарей с описанием машины
    parsed_objects = list()

    for i in range(1, count_pages + 1):
        user_agent = UserAgent(use_cache_server=False).chrome
        response = requests.get(get_items_url(i), headers={'User-Agent': user_agent})
        if response.ok:
            for item_url in parse_items(response.content):
                response = requests.get(item_url, headers={'User-Agent': user_agent})
                if response.ok:
                    try:
                        parsed_objects.append(parse_item(response.content))
                    except:
                        print(f"Error on {item_url}")
                        raise
        print(
            f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}, Обработано страниц: {i}\nОбработано объектов: {len(parsed_objects)}")
        sleep(2)
    return parsed_objects

# Запуск парсера

In [24]:
# соберем данные с первых 20 страниц
data = get_data(20)

2019-11-16 23:10:24, Обработано страниц: 1
Обработано объектов: 20
2019-11-16 23:10:40, Обработано страниц: 2
Обработано объектов: 40
2019-11-16 23:10:56, Обработано страниц: 3
Обработано объектов: 60
2019-11-16 23:11:12, Обработано страниц: 4
Обработано объектов: 80
2019-11-16 23:11:29, Обработано страниц: 5
Обработано объектов: 100
2019-11-16 23:11:45, Обработано страниц: 6
Обработано объектов: 120
2019-11-16 23:12:00, Обработано страниц: 7
Обработано объектов: 140
2019-11-16 23:12:16, Обработано страниц: 8
Обработано объектов: 160
2019-11-16 23:12:33, Обработано страниц: 9
Обработано объектов: 180
2019-11-16 23:12:50, Обработано страниц: 10
Обработано объектов: 200
2019-11-16 23:13:05, Обработано страниц: 11
Обработано объектов: 220
2019-11-16 23:13:21, Обработано страниц: 12
Обработано объектов: 240
2019-11-16 23:13:36, Обработано страниц: 13
Обработано объектов: 260
2019-11-16 23:13:53, Обработано страниц: 14
Обработано объектов: 280
2019-11-16 23:14:09, Обработано страниц: 15
Обр

In [25]:
data_df = pd.DataFrame(data)

In [26]:
data_df.head()

,Цена,Год,Модель,Двигатель,Мощность,Трансмиссия,Привод,Пробег,Руль
0,380000,2011,Лада 4x4 Бронто,"бензин, 1.7 л",81,механика,4WD,41000,левый
1,765000,2008,Mitsubishi Lancer,"бензин, 2.0 л",241,автомат,4WD,189000,левый
2,805000,2013,Hyundai ix35,"бензин, 2.0 л, ГБО",150,автомат,4WD,71000,левый
3,345000,2008,Hyundai Getz,"бензин, 1.4 л",95,автомат,передний,138000,левый
4,325000,2007,Opel Astra,"бензин, 1.8 л",140,механика,передний,,левый


In [27]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
Цена           400 non-null object
Год            400 non-null object
Модель         400 non-null object
Двигатель      400 non-null object
Мощность       400 non-null object
Трансмиссия    400 non-null object
Привод         400 non-null object
Пробег         400 non-null object
Руль           400 non-null object
dtypes: object(9)
memory usage: 28.2+ KB


In [28]:
data_df.to_csv('drom.csv')